In [5]:
!pip install pennylane

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 61.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 56.3 MB/s eta 0:00:00:00:01


A quantum graph neural network can be implemented by introducing a Hamiltonian that incorporates information about both nodes and edges. Specifically, single-qubit rotations can be employed to embed node features, while two-qubit operators can be utilized to embed edge features. In the simplest case, we can introduce an Ising model Hamiltonian, where an edge exists between neighboring nodes.

In [7]:
import pennylane as qml
from pennylane import numpy as np

In [8]:
dev = qml.device("default.qubit", wires=4)
n_wires = 4
n_layers = 5

graph = [(0, 1), (0, 3), (1, 2), (2, 3)]

# unitary operator U_B with parameter beta
def Nodes(x):
    for wire in range(n_wires):
        qml.RX(x[wire], wires=wire)


# unitary operator U_C with parameter gamma
def Edges(g,graph):
    for edge in graph:
        wire1 = edge[0]
        wire2 = edge[1]
        qml.CNOT(wires=[wire1, wire2])
        qml.RZ(g[wire1, wire2], wires=wire2)
        qml.CNOT(wires=[wire1, wire2])

@qml.qnode(dev)
def Task_4(X,G,graph):

    for wire in range(n_wires):
        qml.Hadamard(wires=wire)

    for i in range(n_layers):
        Edges(G[i],graph)
        Nodes(X[i])

    return qml.expval(qml.PauliZ(0))

In [9]:
X = np.random.uniform(size = (n_layers,n_wires))
G = np.random.uniform(size = (n_layers,n_wires,n_wires))
Net = Task_4(X,G,graph)
circuit1_1 = qml.draw(Task_4)(X,G,graph)
print("\nCircuit1:")
print(circuit1_1)


Circuit1:
0: ──H─╭●───────────╭●─╭●───────────╭●──RX(0.53)────────────────────────╭●─────────────────
1: ──H─╰X──RZ(0.10)─╰X─│────────────│──╭●──────────────────╭●──RX(0.11)─╰X─────────RZ(0.23)
2: ──H─────────────────│────────────│──╰X─────────RZ(0.13)─╰X─╭●──────────────────╭●───────
3: ──H─────────────────╰X──RZ(0.60)─╰X────────────────────────╰X─────────RZ(0.07)─╰X───────

──╭●────────╭●───────────╭●──RX(0.60)────────────────────────╭●──────────────────╭●────────╭●
──╰X────────│────────────│──╭●──────────────────╭●──RX(0.74)─╰X─────────RZ(0.62)─╰X────────│─
───RX(0.59)─│────────────│──╰X─────────RZ(0.41)─╰X─╭●──────────────────╭●─────────RX(0.44)─│─
───RX(0.05)─╰X──RZ(0.33)─╰X────────────────────────╰X─────────RZ(0.61)─╰X─────────RX(0.67)─╰X

────────────╭●──RX(0.09)────────────────────────╭●──────────────────╭●────────╭●───────────╭●
────────────│──╭●──────────────────╭●──RX(0.83)─╰X─────────RZ(0.46)─╰X────────│────────────│─
────────────│──╰X─────────RZ(0.52)─╰X─╭●───────────────